In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

df = pd.read_csv("messages.csv")

df['text'] = (df['subject'].astype(str) + " " + df['message'].astype(str)).fillna('')

labels = df['label'].values


In [5]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

inputs = tokenizer(df['text'].tolist(), padding=True, truncation=True, max_length=512, return_tensors='tf')

input_ids = inputs['input_ids'].numpy()

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.2, random_state=42
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:

wolves = 10
dimensions = train_inputs.shape[1]
iterations = 50

solutions = np.random.rand(wolves, dimensions)

def fitness_function(solution):
    selected_features = np.nonzero(solution > 0.5)[0]
    if len(selected_features) == 0:
        return 0
    return len(selected_features)
for iter in range(iterations):
    fitness_values = np.array([fitness_function(sol) for sol in solutions])
    sorted_indices = np.argsort(fitness_values)[::-1]
    alpha, beta, delta = solutions[sorted_indices[:3]]
    for i in range(wolves):
        A = 2 * np.random.rand(dimensions) - 1
        C = 2 * np.random.rand(dimensions)
        D_alpha = abs(C * alpha - solutions[i])
        X_new = alpha - A * D_alpha
        solutions[i] = np.clip(X_new, 0, 1)

best_solution = solutions[np.argmax(fitness_values)]
selected_features = np.nonzero(best_solution > 0.5)[0]

train_inputs_selected = train_inputs[:, selected_features]
val_inputs_selected = val_inputs[:, selected_features]

print(f"Selected {len(selected_features)} important features using GWO.")


Selected 55 important features using GWO.


In [7]:

train_inputs_padded = pad_sequences(train_inputs_selected, padding='post', maxlen=len(selected_features))
val_inputs_padded = pad_sequences(val_inputs_selected, padding='post', maxlen=len(selected_features))


In [8]:

def build_cnn_model(input_dim):
    input_ids = layers.Input(shape=(input_dim,), dtype=tf.int32, name='input_ids')

    embedding = layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=768, input_length=input_dim)(input_ids)

    x = layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(embedding)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=input_ids, outputs=output)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = build_cnn_model(len(selected_features))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)               │ (None, 55)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 55, 768)             │      23,440,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 55, 128)             │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 27, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 27, 128)             │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 13, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1664)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         106,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,891,841 (91.14 MB)

 Trainable params: 23,891,841 (91.14 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:

history = model.fit(
    train_inputs_padded, train_labels,
    epochs=10, batch_size=16,
    validation_data=(val_inputs_padded, val_labels)
)

loss, accuracy = model.evaluate(val_inputs_padded, val_labels)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


Epoch 1/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 58s 385ms/step - accuracy: 0.8343 - loss: 0.4064 - val_accuracy: 0.9620 - val_loss: 0.1190
Epoch 2/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 83s 397ms/step - accuracy: 0.9830 - loss: 0.0532 - val_accuracy: 0.9672 - val_loss: 0.1255
Epoch 3/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 81s 392ms/step - accuracy: 0.9978 - loss: 0.0067 - val_accuracy: 0.9430 - val_loss: 0.3356
Epoch 4/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 58s 398ms/step - accuracy: 0.9971 - loss: 0.0059 - val_accuracy: 0.9637 - val_loss: 0.2351
Epoch 5/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 93s 478ms/step - accuracy: 0.9945 - loss: 0.0144 - val_accuracy: 0.9672 - val_loss: 0.1574
Epoch 6/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 74s 421ms/step - accuracy: 0.9985 - loss: 0.0028 - val_accuracy: 0.9655 - val_loss: 0.1910
Epoch 7/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 82s 422ms/step - accuracy: 0.9987 - loss: 0.0018 - val_accuracy: 0.9672 - val_loss: 0.2439
Epoch 8/10
145/145 ━━━━━━━━━━━━━━━━━━━━ 60s 412ms/step - accuracy: 1.0000 - loss: 4

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


y_pred_prob = model.predict(val_inputs_padded)
y_pred = (y_pred_prob > 0.5).astype(int)


precision = precision_score(val_labels, y_pred)
recall = recall_score(val_labels, y_pred)
f1 = f1_score(val_labels, y_pred)

print(f"Validation Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1-score: {f1*100:.2f}%")



19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
Validation Accuracy: 94.82%
Precision: 84.55%
Recall: 90.43%
F1-score: 87.39%
